# Linear Network

The `nengolib.LinearNetwork` class is a `nengo` network that abstracts away many of the details of Principle 3 from the NEF. Simply supply a `LinearSystem` that you would like to simulate, and the network will map its dynamics onto a recurrently connected ensemble using a given `synapse`. This can be understood equivalently as convolving a causal filter with some arbitrary input signal. 

### Delay Example

For example, the `nengolib.synapses.PadeDelay` is a linear system that we can build into a biologically plausible population of neurons to delay an input signal by some fixed amount of time.

In [ ]:
%pylab inline
import pylab
try:
    import seaborn as sns  # optional; prettier graphs
except ImportError:
    pass

import numpy as np
import nengo
import nengolib
from nengolib.synapses import PadeDelay

delay = 0.1
T = 2.0
dt = 0.001

with nengolib.Network() as model:
    stim = nengo.Node(output=nengo.processes.WhiteSignal(T, high=8))
    
    # Build a LinearNetwork that approximations a delay
    subnet = nengolib.networks.LinearNetwork(
        PadeDelay(delay, order=4), n_neurons_per_ensemble=200, synapse=0.02,
        radii=1.0, dt=dt, output_synapse=0.02)
    nengo.Connection(stim, subnet.input, synapse=None)

    # Add some probes
    p = nengo.Probe(subnet.output)
    p_stim = nengo.Probe(stim)
    
sim = nengo.Simulator(model, dt=dt)
sim.run(T)

And we can visualize the difference between the ideal shifted input and the actual output of the network to see how accurately it performs. 

In [ ]:
offset = int(delay / dt + 1)

pylab.figure()
pylab.title('Delayed Input Signal')
pylab.plot(sim.trange()[offset:], sim.data[p][offset:], label="Actual")
pylab.plot(sim.trange()[offset:], sim.data[p_stim][:-offset], label="Expected")
pylab.legend()
pylab.xlabel('Time (s)')
pylab.show()

### Setting Radii

One of the most subtle but difficult aspects of building these networks is making sure that the representational range that the neurons are optimized over (the eval points and the radius in Nengo), matches the states that the network will actually visit.

Crucially, this depends on how characteristics of the input signal relate to the given dynamics. By default, the `LinearNetwork` class plans for the worst-case input within the given radius, and accordingly plays safe by over-estimating the range of states that the neurons should represent.

If we have more information about the problem, say by having looked at the responses of the states for typical inputs, we may use that to shrink the radii.

In [ ]:
with model:
    p_x = nengo.Probe(subnet.x.output, synapse=0.01)

sim = nengo.Simulator(model, dt=dt)
sim.run(T)

pylab.figure()
pylab.title('States')
pylab.plot(sim.trange(), sim.data[p_x])
pylab.xlabel('Time (s)')
pylab.show()

radii = np.max(abs(sim.data[p_x]), axis=0)
print radii

More information on setting radii will be added here in the future, as well as simulating networks with other synapses on both analog and digital hardware.

### Input and Output Filtering

By default the network will not filter the input or output signal. But an LTI normally filters the input signal, which you can do by specifying the same `input_synapse`. By linearity this is equivalent to filtering the output, which you can also specify through the `output_synapse`. Note that even though the output is equivalent between these two options, the resulting state-space is not. Therefore depending on the frequency of the input signal, you may need to reconsider the radius when switching between these options.

Note that since some systems will have a passthrough (`D != 0`), you should default to never filtering the connections to/from the input/output nodes. You should always try to pass the filters to the `LinearSystem`, which will apply the filter without the passthrough for you. If the system does not have a passthrough, then it doesn't matter where the remaining filter goes (as long as it only appears once on either the input or the output).